# Elektron WebSocket API MRN Example with Python

## Machine Readable News Overview

Refinitiv Machine Readable News (MRN) is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news and news analytics directly to your applications. This enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies and manage event risk.

MRN aims for replacing the legacy News 2K (N2_UBMS and N2_STORY).

### MRN Data behavior
MRN is published over Elektron using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed, and potentially fragmented across multiple messages, in order to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:
* RSSL connection (ESDK [C++](https://developers.refinitiv.com/elektron/elektron-sdk-cc)/[Java](https://developers.refinitiv.com/elektron/elektron-sdk-java)): BUFFER type
* WebSocket connection: Base64 ascii string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into a number of fragments (BUFFER or Base64 ascii string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/mrn_process.png"/>

Therefore, in order to parse the core content data, the application will need to reverse this process. The WebSocket application also need to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field.

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2 and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

Please see a full documentation of this example application in [this article](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:
* [Webinar Recording: Introduction to Machine Readable News](https://developers.refinitiv.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [2]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib

In [4]:
# TREP connection variables

hostname = '172.20.33.30'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [5]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_news_messages = []

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_STORY*: Real-time News
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_TRSI*: News Sentiment Indices

 

In [6]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_TRNA'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [7]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [8]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _news_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [9]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [10]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(30)
    web_socket_app.close()

Connecting to WebSocket ws://172.20.33.30:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"10.42.86.37"
    },
    "Name":"root"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":61430,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"10.42.86.37",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":1,
        "SupportPauseResume":1,
        "SupportStandby":1,
        "SupportViewRequests":1
      },
      "Name":"root"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Open",
      "Text":"Login accepted by hos

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/50Va3OiSPCvUHxdk2JATeQbAgq7OhhAuWRr62oCo2HlkfAwa7b879szg6fJ7blVZ1ndTT+ne7p7fsqkINm+SeNa1n+ePoK4rCiwvgKvrmljZoBkXTbnC3wv9zpmmXAdeaEPlL6iaIPBjQZCXx/7xuo6CM/ptfytM3MTcPTOgLMxySkIxhXZpQmRnDJL0mIjGWNQeKzKLa2MuEnLAnSW2LInLrYtEK3Tqm7mtGCiABAtYnCDenKWFluauInIIk0WZZ0Ke+UkBGdNpUfbSn356/tb/LeqKCrS1GgfJImz9r98GrbL1jfWq0mznb+93Dj5nuTzT8Xz+O2zfICMinJHs2Zvlm3RdJEamvNPforXtEjKVwiDVEc+9P5TrPYvijXrknRwUXpj8ZM+V2lMQ1Jt4AaKJI1JU1YfalnRjO6IKOC1wn49uWalzQF0LXCFzniYbkiT7kBfub5FCN1o74RtU5GMyZA6Gg3PnYnLEIbK4PZ2OOyfSaOySrpE1NuevCuzNqe/L/HocolHfyjx8HKNhxeLrGofq8w7gr7WLiixeXosk32QvlHec3GZP5Nif7oh+iN+IsWG/ravnyhJoEvhxjYgAUZOqi2buZxViFR7WV+TrKaibKzrz67+vH4H0NjXcKAVrWoRJwx0mL6hgpAKOcFsiDGYRFg1TGQNuzFAZvAjHG8W5qbCtPb9xTR/aD9P7u5W3g4VxPGMsTJ7lN+nnJCGhPtnNsoY2GxCKU0mJE+zPVsYPNRZfmzkYUO4liE53sxy8RRGXrqSpMiWHGPFgCXNvMj2pYXvWUszdD0swT/w5rbkTSRv6U

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/51V3XOiSBD/Vyhe7mGT3DAiBt6IDIYtBReIrl5dXU1gkpDwYfjQsFv+79fDkFVzOa/qfOhf218z3dPd/JRpTtO2TqJKNn4e/gRRUTIQ/QGyqmL1OAWQDXk8m7sr+aIXFnFnI88NFevDa01ThzoofSOchsri6pac/HmQ/+wdnRhCnbh0YpdmDBQhS1lVM8lrn0FzXxYvrDSjOilyUN65FrEdl1igekjKqp6xnKsCAJZH4K9cyGmSv7DYiUUCSTwvqkT4o4MSgtWlYS9dbI4Va/gXRggrA+ULaiL9Zokn7er3+W6XLrw8Lf1CJVsncO6DjePr81beX5zGVT7GnbhkWFdDFPdxMVKHcVPeVd5NWNtlVXpvG4c0zdvz67Qu34h5p2uxI++hRHmxZWndjosmr/sMapZ1f2UDX8i7JI+LHRyj4Ftxkc/VWD2rHljntMOz2pHV3XRTJhELafkIT5rHSUTrovzwRiU85paKh7lC/HchV/zJMiB9Vx2LXPZI62QL5uhKUXWs6SfKpi5pynWaog3VY0fxFsIRD1RFx0fKZVHG72mA07ZIm4x9XmDtfIG18wXWzhZYO1vg0ccCd83AdpUDNnw674u4DZIfrKtYVGQbmre9L/Qfe4ueaP7IPh2VJ0ZjaFB4rEfQgCCj5Quf4IyXh5atbDzQtGKiZnyQDoF3x8Xbg0UL05ktWFmJc8LAUBHSkKJgSAnG4l8my89ce+vb6+9rN54MX+koXk2/IhvPJvrry5et+iOaTV7l05RjWtOw3fC14IKYDz1jsU2zJG35+umOOsqPX4dMSRASyVt9lS4lyyOB5H

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/+1Zy27bRhT9FYNr2yBn+NROpmi9aUWk7ThFUdDkWKItkQofTpXAgJOmiyBNW3SRRRdFuumii6LdFUWLfozjNPmLDmeGpqRYNGojQBHYC97DuZfzOHPmamb8iHMCZzRNfDfmKo+KF8sNI4SLPsFlcYwSfYQNV+H0bs/c41ZZYeiRGK5XEYEmqYrCayJ29iuaporrNoHWpr2xud5rc5+yr5oermcunhSbzhhhhxUeJBtOcLRSj8J0sqKH0QRH7EfhEYqqbuKHAQ7aNmvGZtM0ath14Edx0kVB5rKwQYGL65FWuZEfHCGv6dFR+F4vjH36PV84cWVJVGkfutv1w8/wH+B5IABg25tBZ7fVF473PP3hEa/p7qEq9WCr2YfWDpJ7ra414E7wkILwGI2SqR6mQcJaStCYvJKGHviBFz7AzQigwZ2sLnUD8T23MOOGtTKvVOpVaqSnk8h3ke1EAzwFgee7ThJGC1xGaISOHUIgvy6KChDEVS7OuB3jB9PAmjBTZqKBk/jH5AMF4FkFc840iZxR5hOghGd8xkdng34oiLwowBnnbhh5bCCqvModh6N0jC6nWCynGIrlHMtSKckKKGeZf4/nLOK/rBgIebJMoMgrbMVUG21rb9mKofGzK6YaO0N/pe3EyL/2ctHml8uH1TVfSjlfyjj/4XS9TNa8AlVNli/XNZQlBS7TtSTzUNYu17V4ha6voBiUUwxLKYblooY31LQqAUiErPJQYpq2t4zOMk2z+FlN2+HRNFwxRshNojBY6STerao/BlUL5RQL5RSLpRSLpRRLNxO1yotinqihnG9ttrd6S0

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/51WW3OqSBD+KxSvMdkBFZQ3ImhIFBAwHN3a2kIYDZGL4aIHT/nfTw9DouZkTdXmYfqjv+me6Z7uNr9YL/Giqgj9nJV+nT5sP80wqP4GXZ7jYhCBYCV2MDH1OdtqlGlQ72FNqcP3uz1B6HT7QFqSM3a457sH9eJjxf7TGGoBuLowqdW6F2MgHBzhvMCMUb0Cs8zSDc5kvwjTBMiZrqhDTVcVoFZhlhcTnBDKBoETH+y5FhuFyQYHWkADCAMzzUNqj04kOCsyaejqvDzglO6/PEI81+aWgruxh2/L3MD+PHLfdofFIe244nNffnLG06c42yeYPbYu/XLf+n3bbB931VTRzIeevnrYuTu+knV0c7hpr36u7Zm7bncOf/jlv/V7g0q/f+/yo2r+l7nfR89GEmVW2lF3mq0t7a1m9c2KPULqk3SHo6IapGVSNJkpcFx/slK7xe7DJEj3cAzHP9CLfE3znat0W7nGdq+yolLfdJuFPna8bA2lkgSh7xVp9untMyiSnUcf/A6Rvxabk1KIYWmq9Vyl47VXhDvYju44sSeI/QuyLDIvIpyA+pzQPePoW1BDnuO5rnBGumkWNGFwvRa7S6Myxl8nWLyeYPF6gsWrCRavJrj3OcF1MeB9rsEe0vXLNKjs8IDrjPlpvPWS6j2qFot/+i9essZftuAL9gIoUHisNTCgiL1sQyZDTNLjZRUrrbwoxzRnpEFPjvfnyTvCjgq6Pn7GWU7PcWypg5CAOI6HkKAt/qMDdGOp+pPVwUq73fn2dWhNt45pyPfLxFnFg7x0nhbmiL0MOfAKz6m2ZNzooCbDBO

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/51VbZOiOBD+Kxaf7uochyTKuHxDQdfdMbrgjOtcXV1FyDjsIGgAXWbL/36dBFfdmvOqjg/ppl/o9NMv/DBYypKqiMPcsH+cXoIwExxEf4Isz3nRT4AYttEfT+nCaNbCLFI2xtTumBax2h2z2wGlb9PFYNIKZsZfteUoAt8LGyWmbM1BQavnLGcNpwfypcheuXDCIs5SUD1Q1xuMqOeC6jkWeTHmqVQFQHgagjdqGkmcvvJoFOn7xtE0y2Ptb56U8LFC2PNXQWbBIF3+jU0TI4L/6H54wyMyDp/Gu3LZfhYk+I62OLuNSrbofopv/UlnaxwgkTTb8aSo+lmZFnWkgq/Vq7rFPk6jbA9hEP5oHJr/qsbtq2riXtN2rmrvXHXTjYhDPmNiBcCnURyyIhO/YCl4wndMA9gy5dM0cgntGo662OciylesiHdgbrZMbFoY3V1oy0KwRCo/YILbnTOdLkbt2bEQIeffnWciOiaCm8YuS8o1fx9ifB1ifB1ichVichXiu18hVu3A9/kIbOTYLLOoCuI3rjALs/WGpdWpPPx7+MLSFX+3qV84i6BFoVwr0IBgzcSrHK21hIeJyrCfWZJzjZls+bO6n4N3AIsqhws9cpHrOLPAbpumZSKEISUYDD0DgznFTh+5vXoG0KC83bJP1d5ZhcPPH7eusxx6375u3orEx5vRYnX/sie+cZlyxAo2qzZqfEEsx5PzaMDWcVLJvaBCneUn7WApBA6MeeOm8aXdGNH+ZOw1G+PA+9wgXdL4jaC738ElAbBKtpIuPJWI8ILJcGpBJXqXpD+TkLukjGKJjNpFdG

RECEIVED: 
[
  {
    "Type":"Ping"
  }
]
SENT:
{
  "Type":"Pong"
}
RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/51VbZOiOBD+KxRf17EI+IJ8i4DKqYEFHHe8urrKQMZhRfB40fW2/O/XIbiru3te1fEh/STd6aSfdDdfZZrR9FwlUSkbX79PgigvGCz9DmtlySozBSEbsrn0yIvcaRfzuLGRPaOnjvr6YNBDI1D6xmw1Q8/dmX03eROz5wXMJvIfrRMnBrd325tlQvcMFLP6vaJ7uksk9/wZdK9FvmMFjqokz0C9IpY9cYhtgeotKcpqyTKuCkCwLAIPqCOnSbZjsRPzq8KpWX5kaXU28zqrRIRJxfbNVDaUjnxKsjg/gXOkzuRL51/Vau+hWrMeafsPtUPQwk0PRRKxkBZbYCmLk4hWefFD0AVL2ZGKSLsK/zpyyTnYw9A+GrpZImxLq+QI5kpXUYd6XxveaeuqoClXItRX+4MbnZeXyXWn3h/oo9uT1nkRt3GoWkc+5mm9Z79mWH/MsP6YYf0hw/pDhtFPFHOSM3YqHTDi6f+ax+cg+Zs1rxHl+wPNztfNHZl9id5ptmW/zL53RmNINHiuLWhgYU+LHS+RPeeHFmfZeKNpyQRpPDe/Oz7dsncBi3MJF3pmRSnOCQOjpygDBSEVYpITXjFVYUzWRMUmsrQ/VUVRkYaCj6fhzqrLfPzhwwbn89VuN98dlmsyWqRYmSPfKdKTfB9yTCsang+81ggs8zpiLJ7QfZKeeX03R93Ex2tyNQvxEs8dyX35TXqSpq5rScADJqaDF5LnBk7ouETCxJLw2Fk44YsUutL

RECEIVED: 
[
  {
    "DoNotCache":true,
    "DoNotConflate":true,
    "Domain":"NewsTextAnalytics",
    "Fields":{
      "ACTIV_DATE":"2020-02-13",
      "FRAGMENT":"H4sIAAAAAAAA/51VUXPaOBD+Kx6/ljCSDSH2m7FFcAvCtQUEbjo3iq0QB2OntiGlHf57VxZcoNejM+cH7UrfaqX9tLv+ofOcZ/s6jSvd/vE+ieKiFLD0F6xVlajdDIRu6+44oAu9dVwsksZGD+yOYXXvbm872AIwtIfTIZ61h+Ri8qRmsxHMBvqXoxM/AbcX25tlyjcCgOH2ueYbvk61yf4FsMeyWIvSieu0yAGeUo8MfEo8gJ7SsqrHIpdQBELkMXjALT1L87VI/EReFU7Ni53I6r1bbPNaRZjWYtNMdRu19Lc0T4o3cI6NoX5o/SdsdK7CpncN7V5Fe4DCTV/LNBaMlytgKU/SmNdF+UvQpcjEjqtI20h+Lb2SHGxgOD4aPluiYsXrdAfmqI2A9A6+RLd1yTMJYtxD3d4ZFhRVetp51zU7yDgD50WZHOMwui19V2Tbjfg9w9Z1hq3rDFtXGbauMixDvaRYkpyLt8oHI5n+j0Wyj9LvonmNuNi88nx/2tzSxbf4mecr8dvsexY8gUSD51oBAgsbXq5liWwkP7zc6/YTzyqhSJO5+e747Zy9A1jsK7jQTJSVOodFdgehW4SxATHpqayYurQHc2o4LvbMvw2EDGziXjT98DK0PtPvxnASmH1UJOx1GfMPT6X/lT18u++WTOiXISe85mz/KmuNwrKsIyGSAd+k2V7Wd3PUWXyyJqdD5oydT742WXzUbjTXCXzmjDTyEBDq+WwaEs2PmqnLiKexidYnmhME4eTBHzuMjBaaw7TIGRNtRGZkpDmR5lPNQNjS5j4bamPn4yT02UKbDACYkYiNCWWR5vnhu8

WebSocket Closed


In [11]:
_news_messages

[{'analytics': {'analyticsScores': [{'assetClass': 'CMPNY',
     'assetCodes': ['P:4295885727', 'R:AKAS.OL'],
     'assetId': '4295885727',
     'assetName': 'Akastor ASA',
     'brokerAction': 'UNDEFINED',
     'firstMentionSentence': 1,
     'linkedIds': [{'idPosition': 0,
       'linkedId': 'tr:Wkr2n4tXB_2002132hSYMVYgJihLPaIeKu8aZRdSZJhyx7tPT5+d7Y'}],
     'noveltyCounts': [{'itemCount': 1, 'window': '12H'},
      {'itemCount': 1, 'window': '24H'},
      {'itemCount': 1, 'window': '3D'},
      {'itemCount': 1, 'window': '5D'},
      {'itemCount': 1, 'window': '7D'}],
     'priceTargetIndicator': 'UNDEFINED',
     'relevance': 1.0,
     'sentimentClass': 1,
     'sentimentNegative': 0.0524704,
     'sentimentNeutral': 0.135307,
     'sentimentPositive': 0.812223,
     'sentimentWordCount': 316,
     'volumeCounts': [{'itemCount': 5, 'window': '12H'},
      {'itemCount': 5, 'window': '24H'},
      {'itemCount': 5, 'window': '3D'},
      {'itemCount': 5, 'window': '5D'},
      {'itemC

In [13]:
print("first 10 analytics\n")
for analytic in _news_messages[:10]:
    if analytic["analytics"]:
        print(analytic["analytics"])

first 10 analytics

{'analyticsScores': [{'assetClass': 'CMPNY', 'assetCodes': ['P:4295885727', 'R:AKAS.OL'], 'assetId': '4295885727', 'assetName': 'Akastor ASA', 'brokerAction': 'UNDEFINED', 'firstMentionSentence': 1, 'linkedIds': [{'idPosition': 0, 'linkedId': 'tr:Wkr2n4tXB_2002132hSYMVYgJihLPaIeKu8aZRdSZJhyx7tPT5+d7Y'}], 'noveltyCounts': [{'itemCount': 1, 'window': '12H'}, {'itemCount': 1, 'window': '24H'}, {'itemCount': 1, 'window': '3D'}, {'itemCount': 1, 'window': '5D'}, {'itemCount': 1, 'window': '7D'}], 'priceTargetIndicator': 'UNDEFINED', 'relevance': 1.0, 'sentimentClass': 1, 'sentimentNegative': 0.0524704, 'sentimentNeutral': 0.135307, 'sentimentPositive': 0.812223, 'sentimentWordCount': 316, 'volumeCounts': [{'itemCount': 5, 'window': '12H'}, {'itemCount': 5, 'window': '24H'}, {'itemCount': 5, 'window': '3D'}, {'itemCount': 5, 'window': '5D'}, {'itemCount': 5, 'window': '7D'}]}, {'assetClass': 'COM', 'assetCodes': ['N2:MTAL', 'N2:MTAL08'], 'assetId': 'MTAL', 'assetName': 'N

In [41]:
test_analytics = _news_messages[:1]
test_analytics

[{'analytics': {'analyticsScores': [{'assetClass': 'CMPNY',
     'assetCodes': ['P:4295885727', 'R:AKAS.OL'],
     'assetId': '4295885727',
     'assetName': 'Akastor ASA',
     'brokerAction': 'UNDEFINED',
     'firstMentionSentence': 1,
     'linkedIds': [{'idPosition': 0,
       'linkedId': 'tr:Wkr2n4tXB_2002132hSYMVYgJihLPaIeKu8aZRdSZJhyx7tPT5+d7Y'}],
     'noveltyCounts': [{'itemCount': 1, 'window': '12H'},
      {'itemCount': 1, 'window': '24H'},
      {'itemCount': 1, 'window': '3D'},
      {'itemCount': 1, 'window': '5D'},
      {'itemCount': 1, 'window': '7D'}],
     'priceTargetIndicator': 'UNDEFINED',
     'relevance': 1.0,
     'sentimentClass': 1,
     'sentimentNegative': 0.0524704,
     'sentimentNeutral': 0.135307,
     'sentimentPositive': 0.812223,
     'sentimentWordCount': 316,
     'volumeCounts': [{'itemCount': 5, 'window': '12H'},
      {'itemCount': 5, 'window': '24H'},
      {'itemCount': 5, 'window': '3D'},
      {'itemCount': 5, 'window': '5D'},
      {'itemC

In [46]:
analytic_scores_group = test_analytics[0]["analytics"]["analyticsScores"]
analytic_scores_group

[{'assetClass': 'CMPNY',
  'assetCodes': ['P:4295885727', 'R:AKAS.OL'],
  'assetId': '4295885727',
  'assetName': 'Akastor ASA',
  'brokerAction': 'UNDEFINED',
  'firstMentionSentence': 1,
  'linkedIds': [{'idPosition': 0,
    'linkedId': 'tr:Wkr2n4tXB_2002132hSYMVYgJihLPaIeKu8aZRdSZJhyx7tPT5+d7Y'}],
  'noveltyCounts': [{'itemCount': 1, 'window': '12H'},
   {'itemCount': 1, 'window': '24H'},
   {'itemCount': 1, 'window': '3D'},
   {'itemCount': 1, 'window': '5D'},
   {'itemCount': 1, 'window': '7D'}],
  'priceTargetIndicator': 'UNDEFINED',
  'relevance': 1.0,
  'sentimentClass': 1,
  'sentimentNegative': 0.0524704,
  'sentimentNeutral': 0.135307,
  'sentimentPositive': 0.812223,
  'sentimentWordCount': 316,
  'volumeCounts': [{'itemCount': 5, 'window': '12H'},
   {'itemCount': 5, 'window': '24H'},
   {'itemCount': 5, 'window': '3D'},
   {'itemCount': 5, 'window': '5D'},
   {'itemCount': 5, 'window': '7D'}]},
 {'assetClass': 'COM',
  'assetCodes': ['N2:MTAL', 'N2:MTAL08'],
  'assetId': 

In [52]:
for analytic_score in analytic_scores_group:
    if analytic_score["assetClass"]:
        print("assetClass: ", analytic_score["assetClass"])
        print("assetCodes: ", analytic_score["assetCodes"])
        print("Broker Action: ", analytic_score["brokerAction"])
        print("Rel: ",analytic_score["relevance"])
        print("Sent: ", analytic_score["sentimentClass"])
        print("Pos: ", analytic_score["sentimentPositive"])
        print("Neut: ", analytic_score["sentimentNeutral"])
        print("Neg: ", analytic_score["sentimentNegative"])
        print("Price Target: ", analytic_score["priceTargetIndicator"])
        print("1st Mentn: ", analytic_score["firstMentionSentence"])
        print("--------------------------------------------------------")

assetClass:  CMPNY
assetCodes:  ['P:4295885727', 'R:AKAS.OL']
Broker Action:  UNDEFINED
Rel:  1.0
Sent:  1
Pos:  0.812223
Neut:  0.135307
Neg:  0.0524704
Price Target:  UNDEFINED
1st Mentn:  1
--------------------------------------------------------
assetClass:  COM
assetCodes:  ['N2:MTAL', 'N2:MTAL08']
Broker Action:  UNDEFINED
Rel:  1.0
Sent:  1
Pos:  0.658448
Neut:  0.161535
Neg:  0.180017
Price Target:  UNDEFINED
1st Mentn:  0
--------------------------------------------------------


In [44]:
news_item  = analytic_scores_group = test_analytics[0]["newsItem"]
news_item

{'dataType': 'News',
 'feedFamilyCode': 'tr',
 'headline': 'Akastor ASA: Fourth Quarter Results 2019',
 'language': 'en',
 'metadata': {'altId': 'nPn7p7ntLa',
  'audiences': ['NP:CNR', 'NP:PRN'],
  'feedTimestamp': '2020-02-13T06:32:07.468Z',
  'firstCreated': '2020-02-13T06:32:07.415Z',
  'isArchive': False,
  'takeSequence': 1},
 'provider': 'NS:PRN',
 'sourceId': 'Pn7p7ntLa_2002132gdTiL3i4/qqu0xRFS4mMpj2hK1oYLt0RcZPUz',
 'sourceTimestamp': '2020-02-13T06:32:07.415Z',
 'subjects': ['A:1',
  'B:1020',
  'B:11',
  'B:12',
  'B:161',
  'B:162',
  'B:181',
  'B:19',
  'B:2',
  'B:20',
  'B:219',
  'B:227',
  'B:228',
  'B:229',
  'B:278',
  'B:279',
  'B:282',
  'B:9',
  'E:1',
  'E:4E',
  'E:E',
  'G:2',
  'G:3',
  'G:3N',
  'G:A',
  'G:M',
  'M:1P2',
  'M:1QD',
  'M:1WN',
  'M:32',
  'M:B6',
  'M:Z',
  'R:AKAS.OL',
  'N2:ACCI',
  'N2:BACT',
  'N2:BMAT',
  'N2:BMAT08',
  'N2:CMPELC',
  'N2:CMPNY',
  'N2:ENER',
  'N2:ENFF',
  'N2:ENQSVC',
  'N2:EUROP',
  'N2:EUROPE',
  'N2:INDU',
  'N2:L

In [38]:
print("headline: ",news_item["headline"])

headline:  Akastor ASA: Fourth Quarter Results 2019


## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.thomsonreuters.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Introduction to Machine Readable News with Elektron WebSocket API](https://developers.refinitiv.com/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [Machine Readable News (MRN) & N2_UBMS Comparison and Migration Guide](https://developers.refinitiv.com/article/machine-readable-news-mrn-n2_ubms-comparison-and-migration-guide).
* [Introduction to Machine Readable News (MRN) with Elektron Message API (EMA)](https://developers.refinitiv.com/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Elektron Implementation Guide](https://developers.refinitiv.com/elektron/elektron-sdk-java/docs?content=8736&type=documentation_item).
* [Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor](https://github.com/Refinitiv-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).